### Importing the relevant libraries

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

import statsmodels.api as sm
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LinearRegression, Lasso
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.model_selection import cross_val_score

import warnings
warnings.filterwarnings('ignore')

### Reading data

In [4]:
raw_data = pd.read_csv('Data_for_Model_Building.csv')

In [5]:
df = raw_data.copy()
df

,Unnamed: 0,Job Title,Salary Estimate,Job Description,Rating,Company Name,Location,Headquarters,Size,Founded,...,age,python_yn,R_yn,spark,aws,excel,job_simp,seniority,desc_len,num_comp
0,0,Data Scientist,$53K-$91K (Glassdoor est.),"Data Scientist\nLocation: Albuquerque, NM\nEdu...",3.8,Tecolote Research\n3.8,"Albuquerque, NM","Goleta, CA",501 to 1000 employees,1973,...,47,1,0,0,0,1,data scientist,na,2536,0
1,1,Healthcare Data Scientist,$63K-$112K (Glassdoor est.),What You Will Do:\n\nI. General Summary\n\nThe...,3.4,University of Maryland Medical System\n3.4,"Linthicum, MD","Baltimore, MD",10000+ employees,1984,...,36,1,0,0,0,0,data scientist,na,4783,0
2,2,Data Scientist,$80K-$90K (Glassdoor est.),"KnowBe4, Inc. is a high growth information sec...",4.8,KnowBe4\n4.8,"Clearwater, FL","Clearwater, FL",501 to 1000 employees,2010,...,10,1,0,1,0,1,data scientist,na,3461,0
3,3,Data Scientist,$56K-$97K (Glassdoor est.),*Organization and Job ID**\nJob ID: 310709\n\n...,3.8,PNNL\n3.8,"Richland, WA","Richland, WA",1001 to 5000 employees,1965,...,55,1,0,0,0,0,data scientist,na,3883,3
4,4,Data Scientist,$86K-$143K (Glassdoor est.),Data Scientist\nAffinity Solutions / Marketing...,2.9,Affinity Solutions\n2.9,"New York, NY","New York, NY",51 to 200 employees,1998,...,22,1,0,0,0,1,data scientist,na,2728,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
737,737,"Sr Scientist, Immuno-Oncology - Oncology",$58K-$111K (Glassdoor est.),Site Name: USA - Massachusetts - Cambridge\nPo...,3.9,GSK\n3.9,"Cambridge, MA","Brentford, United Kingdom",10000+ employees,1830,...,190,0,0,0,1,0,na,senior,6162,3
738,738,Senior Data Engineer,$72K-$133K (Glassdoor est.),THE CHALLENGE\nEventbrite has a world-class da...,4.4,Eventbrite\n4.4,"Nashville, TN","San Francisco, CA",1001 to 5000 employees,2006,...,14,1,0,1,1,0,data engineer,senior,6130,3
739,739,"Project Scientist - Auton Lab, Robotics Institute",$56K-$91K (Glassdoor est.),The Auton Lab at Carnegie Mellon University is...,2.6,Software Engineering Institute\n2.6,"Pittsburgh, PA","Pittsburgh, PA",501 to 1000 employees,1984,...,36,0,0,0,0,1,na,na,3078,0
740,740,Data Science Manager,$95K-$160K (Glassdoor est.),Data Science ManagerResponsibilities:\n\nOvers...,3.2,"Numeric, LLC\n3.2","Allentown, PA","Chadds Ford, PA",1 to 50 employees,-1,...,-1,0,0,0,0,1,manager,na,1642,0


### Choosing relevant columns

In [7]:
df.columns

Index(['Unnamed: 0', 'Job Title', 'Salary Estimate', 'Job Description',
       'Rating', 'Company Name', 'Location', 'Headquarters', 'Size', 'Founded',
       'Type of ownership', 'Industry', 'Sector', 'Revenue', 'Competitors',
       'hourly', 'employer_provided', 'min_salary', 'max_salary', 'avg_salary',
       'company_txt', 'job_state', 'same_state', 'age', 'python_yn', 'R_yn',
       'spark', 'aws', 'excel', 'job_simp', 'seniority', 'desc_len',
       'num_comp'],
      dtype='object')

In [8]:
df_model = df[['Rating', 'Size', 'Type of ownership', 'Industry', 'Sector', 'Revenue', 'num_comp', 'hourly', 'employer_provided', 'job_state', 'same_state', 'age',
               'python_yn', 'spark', 'aws', 'excel', 'job_simp', 'seniority', 'desc_len', 'avg_salary']]

In [9]:
df_model

,Rating,Size,Type of ownership,Industry,Sector,Revenue,num_comp,hourly,employer_provided,job_state,same_state,age,python_yn,spark,aws,excel,job_simp,seniority,desc_len,avg_salary
0,3.8,501 to 1000 employees,Company - Private,Aerospace & Defense,Aerospace & Defense,$50 to $100 million (USD),0,0,0,NM,0,47,1,0,0,1,data scientist,na,2536,72.0
1,3.4,10000+ employees,Other Organization,Health Care Services & Hospitals,Health Care,$2 to $5 billion (USD),0,0,0,MD,0,36,1,0,0,0,data scientist,na,4783,87.5
2,4.8,501 to 1000 employees,Company - Private,Security Services,Business Services,$100 to $500 million (USD),0,0,0,FL,1,10,1,1,0,1,data scientist,na,3461,85.0
3,3.8,1001 to 5000 employees,Government,Energy,"Oil, Gas, Energy & Utilities",$500 million to $1 billion (USD),3,0,0,WA,1,55,1,0,0,0,data scientist,na,3883,76.5
4,2.9,51 to 200 employees,Company - Private,Advertising & Marketing,Business Services,Unknown / Non-Applicable,3,0,0,NY,1,22,1,0,0,1,data scientist,na,2728,114.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
737,3.9,10000+ employees,Company - Public,Biotech & Pharmaceuticals,Biotech & Pharmaceuticals,$10+ billion (USD),3,0,0,MA,0,190,0,0,1,0,na,senior,6162,84.5
738,4.4,1001 to 5000 employees,Company - Public,Internet,Information Technology,$100 to $500 million (USD),3,0,0,TN,0,14,1,1,1,0,data engineer,senior,6130,102.5
739,2.6,501 to 1000 employees,College / University,Colleges & Universities,Education,Unknown / Non-Applicable,0,0,0,PA,1,36,0,0,0,1,na,na,3078,73.5
740,3.2,1 to 50 employees,Company - Private,Staffing & Outsourcing,Business Services,$5 to $10 million (USD),0,0,0,PA,0,-1,0,0,0,1,manager,na,1642,127.5


### Get Dummy Data

In [11]:
df_dummy = pd.get_dummies(df_model, drop_first=True).astype(int)
df_dummy

,Rating,num_comp,hourly,employer_provided,same_state,age,python_yn,spark,aws,excel,...,job_state_WA,job_state_WI,job_simp_data engineer,job_simp_data scientist,job_simp_director,job_simp_manager,job_simp_mle,job_simp_na,seniority_na,seniority_senior
0,3,0,0,0,0,47,1,0,0,1,...,0,0,0,1,0,0,0,0,1,0
1,3,0,0,0,0,36,1,0,0,0,...,0,0,0,1,0,0,0,0,1,0
2,4,0,0,0,1,10,1,1,0,1,...,0,0,0,1,0,0,0,0,1,0
3,3,3,0,0,1,55,1,0,0,0,...,1,0,0,1,0,0,0,0,1,0
4,2,3,0,0,1,22,1,0,0,1,...,0,0,0,1,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
737,3,3,0,0,0,190,0,0,1,0,...,0,0,0,0,0,0,0,1,0,1
738,4,3,0,0,0,14,1,1,1,0,...,0,0,1,0,0,0,0,0,0,1
739,2,0,0,0,1,36,0,0,0,1,...,0,0,0,0,0,0,0,1,1,0
740,3,0,0,0,0,-1,0,0,0,1,...,0,0,0,0,0,1,0,0,1,0


### Splitting data into train test splits

In [13]:
X = df_dummy.drop('avg_salary', axis=1)
y = df_dummy['avg_salary'].values

In [14]:
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.2, random_state=42)

### Multiple Linear Regression

In [16]:
X = sm.add_constant(X)

In [17]:
model = sm.OLS(y,X)

In [18]:
result = model.fit()

In [19]:
result.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.709
Model:                            OLS   Adj. R-squared:                  0.640
Method:                 Least Squares   F-statistic:                     10.28
Date:                Mon, 05 May 2025   Prob (F-statistic):           3.87e-95
Time:                        23:12:01   Log-Likelihood:                -3310.4
No. Observations:                 742   AIC:                             6907.
Df Residuals:                     599   BIC:                             7566.
Df Model:                         142                                         
Covariance Type:            nonrobust                                         
=====================================================================================================================
                                                        coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------------------------------------
const                                                39.0233     21.131      1.847      0.065      -2.476      80.522
Rating                                                4.2269      1.785      2.368      0.018       0.722       7.732
num_comp                                              2.2604      0.867      2.608      0.009       0.558       3.962
hourly                                              -36.7694      9.160     -4.014      0.000     -54.759     -18.780
employer_provided                                    31.1941     11.372      2.743      0.006       8.861      53.527
same_state                                            2.3297      2.613      0.891      0.373      -2.803       7.462
age                                                   0.0347      0.030      1.156      0.248      -0.024       0.094
python_yn                                             8.3471      2.524      3.307      0.001       3.391      13.303
spark                                                -0.2573      2.915     -0.088      0.930      -5.981       5.467
aws                                                   0.7120      2.742      0.260      0.795      -4.674       6.098
excel                                                -0.6522      2.268     -0.288      0.774      -5.106       3.802
desc_len                                             -0.0001      0.001     -0.148      0.882      -0.002       0.001
Size_1 to 50 employees                               -1.7969      7.890     -0.228      0.820     -17.293      13.699
Size_10000+ employees                                -6.2441      7.009     -0.891      0.373     -20.009       7.521
Size_1001 to 5000 employees                           3.4067      5.568      0.612      0.541      -7.529      14.343
Size_201 to 500 employees                            -0.3185      5.729     -0.056      0.956     -11.570      10.933
Size_5001 to 10000 employees                         -7.1564      6.691     -1.070      0.285     -20.296       5.983
Size_501 to 1000 employees                           -0.5842      5.882     -0.099      0.921     -12.136      10.967
Size_51 to 200 employees                              1.0925      6.132      0.178      0.859     -10.951      13.136
Size_Unknown                                         17.8684     13.014      1.373      0.170      -7.689      43.426
Type of ownership_College / University                3.8022     17.868      0.213      0.832     -31.289      38.893
Type of ownership_Company - Private                   6.4079      5.650      1.134      0.257      -4.688      17.504
Type of ownership_Company - Public                   10.9599      6.095      1.798      0.073      -1.011      22.931
Type of ownersh

In [20]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

In [21]:
model = LinearRegression()

In [22]:
model.fit(X_train,y_train)

LinearRegression()

In [23]:
y_pred = model.predict(X_test)

In [24]:
MAE = mean_absolute_error(y_test,y_pred)
MAE

18.89971647392543

In [25]:
MSE = mean_squared_error(y_test,y_pred)
MSE

690.8462914717425

In [26]:
RMSE = np.sqrt(MSE)
RMSE

26.283955019588326

In [27]:
R_Squared = r2_score(y_test,y_pred)
R_Squared

0.577588500890428

In [28]:
Adj_R_Squared = 1 - (1 - R_Squared)*(len(y_test)-1) / (len(y_test) - X_test.shape[1] - 1)
Adj_R_Squared

3.976995327057936

In [29]:
cross_val_score(model, X_train, y_train, cv=3, scoring='neg_mean_absolute_error')

array([-20.62783295, -19.39141104, -22.62148233])

In [30]:
np.mean(cross_val_score(model, X_train, y_train, cv=3, scoring='neg_mean_absolute_error'))

-20.880242106149304

### Lasso Regression

In [32]:
lasso_model = Lasso(alpha=0.1)

In [33]:
lasso_model.fit(X_train, y_train)

Lasso(alpha=0.1)

In [34]:
np.mean(cross_val_score(lasso_model, X_train, y_train, cv=3, scoring='neg_mean_absolute_error'))

-19.305286470378103

In [35]:
alpha = []
error = []

In [36]:
for i in range(1,100):
    alpha.append(i/10)
    lml = Lasso(alpha=(i/10))
    error.append(np.mean(cross_val_score(lml, X_train, y_train, cv=3, scoring='neg_mean_absolute_error')))

In [37]:
err = tuple(zip(alpha,error))

In [38]:
df_err = pd.DataFrame(err, columns = ['alpha','error'])

In [39]:
df_err[df_err.error == max(df_err.error)]

,alpha,error
0,0.1,-19.305286


### Random Forest Model

In [41]:
RF_model = RandomForestRegressor()

In [42]:
cross_val_score(RF_model, X_train, y_train, cv=3, scoring='neg_mean_absolute_error')

array([-16.88949495, -15.84722222, -14.77695431])

### Tuning Model

In [44]:
parameters = {'n_estimators':range(10,300,10), 'criterion':('squared_error','absolute_error'), 'max_features':('auto','sqrt','log2')}

In [45]:
gs = GridSearchCV(RF_model, parameters, scoring='neg_mean_absolute_error', cv=3)

In [46]:
gs.fit(X_train,y_train)

GridSearchCV(cv=3, estimator=RandomForestRegressor(),
             param_grid={'criterion': ('squared_error', 'absolute_error'),
                         'max_features': ('auto', 'sqrt', 'log2'),
                         'n_estimators': range(10, 300, 10)},
             scoring='neg_mean_absolute_error')

In [47]:
gs.best_score_

-15.902488431731422

In [48]:
gs.best_estimator_

RandomForestRegressor(max_features='sqrt', n_estimators=230)

### Best Ensembles

In [50]:
tpred_lm  = model.predict(X_test)

In [51]:
tpred_lml = lasso_model.predict(X_test)

In [52]:
tpred_rf  = gs.best_estimator_.predict(X_test)

### Model Evaluation

In [54]:
MAE_LR = mean_absolute_error(y_test,tpred_lm)
MAE_LR

18.89971647392543

In [55]:
MAE_LassoR = mean_absolute_error(y_test, tpred_lml)
MAE_LassoR

19.93124371908307

In [56]:
MAE_RF = mean_absolute_error(y_test, tpred_rf)
MAE_RF

12.587598482637876

### Productionization

In [58]:
import pickle

#### Saving

In [84]:
with open('model_file.p','wb') as model_file:
    pickle.dump(gs.best_estimator_,model_file)

In [86]:
file_name = 'model_file.p'

#### Loading

In [89]:
with open(file_name, 'rb') as pickled:
    data = pickle.load(pickled)
model = data

In [91]:
model.predict(X_test.iloc[[1],:])

array([52.27826087])

In [98]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

In [102]:
X_test.iloc[[1],:]

,Rating,num_comp,hourly,employer_provided,same_state,age,python_yn,spark,aws,excel,desc_len,Size_1 to 50 employees,Size_10000+ employees,Size_1001 to 5000 employees,Size_201 to 500 employees,Size_5001 to 10000 employees,Size_501 to 1000 employees,Size_51 to 200 employees,Size_Unknown,Type of ownership_College / University,Type of ownership_Company - Private,Type of ownership_Company - Public,Type of ownership_Government,Type of ownership_Hospital,Type of ownership_Nonprofit Organization,Type of ownership_Other Organization,Type of ownership_School / School District,Type of ownership_Subsidiary or Business Segment,Type of ownership_Unknown,Industry_Accounting,Industry_Advertising & Marketing,Industry_Aerospace & Defense,Industry_Architectural & Engineering Services,Industry_Auctions & Galleries,Industry_Banks & Credit Unions,Industry_Beauty & Personal Accessories Stores,Industry_Biotech & Pharmaceuticals,Industry_Brokerage Services,Industry_Colleges & Universities,Industry_Computer Hardware & Software,Industry_Construction,Industry_Consulting,Industry_Consumer Product Rental,Industry_Consumer Products Manufacturing,"Industry_Department, Clothing, & Shoe Stores",Industry_Education Training Services,Industry_Energy,Industry_Enterprise Software & Network Solutions,Industry_Farm Support Services,Industry_Federal Agencies,Industry_Financial Analytics & Research,Industry_Financial Transaction Processing,Industry_Food & Beverage Manufacturing,Industry_Gambling,Industry_Gas Stations,Industry_Health Care Products Manufacturing,Industry_Health Care Services & Hospitals,"Industry_Health, Beauty, & Fitness",Industry_IT Services,Industry_Industrial Manufacturing,Industry_Insurance Agencies & Brokerages,Industry_Insurance Carriers,Industry_Internet,Industry_Investment Banking & Asset Management,Industry_K-12 Education,Industry_Lending,Industry_Logistics & Supply Chain,Industry_Metals Brokers,Industry_Mining,Industry_Motion Picture Production & Distribution,Industry_Other Retail Stores,Industry_Real Estate,Industry_Religious Organizations,Industry_Research & Development,Industry_Security Services,Industry_Social Assistance,Industry_Sporting Goods Stores,Industry_Staffing & Outsourcing,Industry_Stock Exchanges,Industry_TV Broadcast & Cable Networks,Industry_Telecommunications Manufacturing,Industry_Telecommunications Services,Industry_Transportation Equipment Manufacturing,Industry_Transportation Management,Industry_Travel Agencies,Industry_Trucking,Industry_Video Games,Industry_Wholesale,Sector_Accounting & Legal,Sector_Aerospace & Defense,Sector_Agriculture & Forestry,"Sector_Arts, Entertainment & Recreation",Sector_Biotech & Pharmaceuticals,Sector_Business Services,"Sector_Construction, Repair & Maintenance",Sector_Consumer Services,Sector_Education,Sector_Finance,Sector_Government,Sector_Health Care,Sector_Information Technology,Sector_Insurance,Sector_Manufacturing,Sector_Media,Sector_Mining & Metals,Sector_Non-Profit,"Sector_Oil, Gas, Energy & Utilities",Sector_Real Estate,Sector_Retail,Sector_Telecommunications,Sector_Transportation & Logistics,Sector_Travel & Tourism,Revenue_$1 to $5 million (USD),Revenue_$10 to $25 million (USD),Revenue_$10+ billion (USD),Revenue_$100 to $500 million (USD),Revenue_$2 to $5 billion (USD),Revenue_$25 to $50 million (USD),Revenue_$5 to $10 billion (USD),Revenue_$5 to $10 million (USD),Revenue_$50 to $100 million (USD),Revenue_$500 million to $1 billion (USD),Revenue_-1,Revenue_Less than $1 million (USD),Revenue_Unknown / Non-Applicable,job_state_AZ,job_state_CA,job_state_CO,job_state_CT,job_state_DC,job_state_DE,job_state_FL,job_state_GA,job_state_IA,job_state_ID,job_state_IL,job_state_IN,job_state_KS,job_state_KY,job_state_LA,job_state_MA,job_state_MD,job_state_MI,job_state_MN,job_state_MO,job_state_NC,job_state_NE,job_state_NJ,job_state_NM,job_state_NY,job_state_OH,job_state_OR,job_state_PA,job_state_RI,job_state_SC,job_state_TN,job_state_TX,job_state_UT,job_state_VA,job_state_WA,job_state_WI,job_simp_

In [ ]:
3	0	0	0	1	34	0	0	0	1	4608	0	0	1	0	0	0	0	0	0	1	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	1	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	1	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	1	0	0	0	0	0	0	0	0	0	0	1	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	1	0
